# Practica 2 - Exploración de grafos de estado

## Ejercicio 1

Usted tiene 2 jarras, una con capacidad de 3 litros y otra con capacidad de 5 litros. Además, se cuenta con un grifo. Las jarras se pueden llenar totalmente, vaciar en el piso y vaciar parcialmente el contenido de una jarra en la otra. Las jarras no tienen ninguna clase de marca. El objetivo es medir exactamente un litro de agua

### 1. ¿Cuáles son los estados del problema?

Los estados del problema se pueden definir de acuerdo a la cantidad de agua en cada jarra. En este escenario, definiremos que cada estado es una variación de un litro en cada una de las jarras. Definiendo la jarra de capacidad 3 como $J_3$ y la jarra de capacidad 5 como $J_5$, tenemos el siguiente espacio de opciones:

$$J_5 = \{0,1,2,3,4,5\}$$
$$J_3 = \{0,1,2,3\}$$